# Updating `is_commuting` function to accept opmath instances

Currently, the `is_commuting` function does not accept the following operators: `prod`, `sprod` and `sum`. 

The goal is to remove this limitation and accept those operators.

## Implementation Details

In order to make that work:

In pennylane/ops/functions/is_commuting.py:
- Remove `Prod`, `Sum`, `SProd` from the unsupported_operations list.
- Update the logic in is_commuting to support those operations.
- Avoid using matrix multiplication where it is not necessary


In [54]:
import pennylane as qml

from pennylane.pauli.utils import is_pauli_word

In [8]:
def intersection(wires1, wires2):
    r"""Check if two sets of wires intersect.

    Args:
        wires1 (pennylane.wires.Wires): First set of wires.
        wires2 (pennylane.wires.Wires: Second set of wires.

    Returns:
         bool: True if the two sets of wires are not disjoint and False if disjoint.
    """
    return len(qml.wires.Wires.shared_wires([wires1, wires2])) != 0

@pytest.mark.parametrize(
        "pauli_word_1,pauli_word_2,wire_map,commute_status",
        [
            (qml.Identity(0), qml.PauliZ(0), {0: 0}, True),
            (qml.PauliY(0), qml.PauliZ(0), {0: 0}, False),
            (qml.PauliX(0), qml.PauliX(1), {0: 0, 1: 1}, True),
            (qml.PauliY("x"), qml.PauliX("y"), None, True),
            (
                qml.PauliZ("a") @ qml.PauliY("b") @ qml.PauliZ("d"),
                qml.PauliX("a") @ qml.PauliZ("c") @ qml.PauliY("d"),
                {"a": 0, "b": 1, "c": 2, "d": 3},
                True,
            ),
            (
                qml.PauliX("a") @ qml.PauliY("b") @ qml.PauliZ("d"),
                qml.PauliX("a") @ qml.PauliZ("c") @ qml.PauliY("d"),
                {"a": 0, "b": 1, "c": 2, "d": 3},
                False,
            ),
        ],
    )
    def test_pauli_words(self, pauli_word_1, pauli_word_2, wire_map, commute_status):
        """Test that (non)-commuting Pauli words are correctly identified."""
        do_they_commute = qml.is_commuting(pauli_word_1, pauli_word_2, wire_map=wire_map)
        assert do_they_commute == commute_status

In [66]:
qml.operation.enable_new_opmath()
# qml.operation.disable_new_opmath()
pauli_word_1 = qml.PauliX(0) @ qml.Hadamard(1) @ qml.Identity(2)
pauli_word_2 = qml.PauliX(0) @ qml.PauliY(2)
wire_map = {"a": 0, "b": 0, "c": 2, "d": 3}
assert is_pauli_word(pauli_word_1)
assert is_pauli_word(pauli_word_2)
type(pauli_word_1), type(pauli_word_2)

AssertionError: 

In [67]:
type(qml.PauliX(0) + qml.Hadamard(1) @ qml.Identity(2))

pennylane.ops.op_math.sum.Sum

In [60]:
pauli_word_1

4 * ((Z('a') @ Y('b')) @ Z('d'))

In [61]:
qml.is_commuting(pauli_word_1, pauli_word_2, wire_map=wire_map)

True

In [62]:
intersection(pauli_word_1.wires, pauli_word_2.wires)

True